In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import os
from backup_mongodb import DataAPI

In [2]:
# 获取etf数据
path = os.getcwd()
ETF_data = pd.read_csv(path+'/50ETF行情.csv', encoding='gbk').iloc[:, 1:]
div_ad = pd.read_csv(path+'/50ETF分红调整.csv', encoding='gbk').iloc[:, 1:]
div_ad.set_index('effectDate', inplace=True)
ETF_data.set_index('endDate', inplace=True)
fund_data = pd.concat([ETF_data['NAV'], div_ad['dividendAfTax']], join='outer', axis=1).fillna(0)
fund_data.columns = ["净值", "分红"]
fund_data.index = [i.replace("-", "") for i in fund_data.index]

In [3]:
# 获取期权数据
ip_config = "mongodb://192.168.1.90:27017/"
client = DataAPI(ip_config)
option_data = client.get_day_equity_option_market_data("510050").set_index("TradingDay")

ServerSelectionTimeoutError: 192.168.1.90:27017: [Errno 51] Network is unreachable, Timeout: 30s, Topology Description: <TopologyDescription id: 62c53df8501aa2b50c4a4493, topology_type: Unknown, servers: [<ServerDescription ('192.168.1.90', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('192.168.1.90:27017: [Errno 51] Network is unreachable')>]>

In [11]:
def first_day_build(option_data, fund_data):
    index_df = []
    first_date = option_data.sort_index().index[0]

    # 第一天的期权套保实现与资产配置计算
    data = option_data.loc[first_date]
    data = data.loc[~data["IsCall"]]
    data = data.loc[data["ExpirationDate"] == pd.unique(data["ExpirationDate"])[0]] # 行权日为第一个月的期权数据
    data = data.sort_values("StrikePrice")
    expir_date = data["ExpirationDate"].iloc[0] # strikeprice最低的合约的行权日期

    stock_price = fund_data.loc[first_date, "净值"] # ETF Price
    threshold = stock_price * 0.95 # the threshold of strike price  (<= 0.95*stock_price)

    tmp = data.loc[data["StrikePrice"] <= threshold] # 行权价格低于threshold的第一天的合约



    # 建仓
    if len(tmp) > 0:
        tmp = data.loc[data["StrikePrice"] <= threshold]
        tmp = tmp.sort_values("StrikePrice").iloc[-1]

    else:
        tmp = data.sort_values("StrikePrice").iloc[0]

    stock = stock_price * 10000
    cash = fund_data.loc[first_date, "分红"] * 10000
    option = tmp["Premium"] * tmp["Unit"] # 在下一步换仓时得改变
    price = stock + option# portfolio value
    symbol = tmp["InstrumentID"] # 建仓的合约
    expir_price = tmp["StrikePrice"] # 行权价格
    Unit = 10000

    index_df.append([first_date, fund_data.loc[first_date, "净值"], stock, cash, symbol, expir_price, Unit, option, price])

    # 持仓至下一个结算日前一天 资产变化情况
    for date in fund_data.loc[first_date:expir_date].iloc[1:-1].index:
        stock = fund_data.loc[date, "净值"] * 10000
        cash = fund_data.loc[date, "分红"] * 10000
        data = option_data.loc[date]
        tmp = data.loc[data["InstrumentID"] == symbol].iloc[0]
        option = tmp["Premium"] * tmp["Unit"]
        price = stock + option
        symbol = tmp["InstrumentID"]
        expir_price = tmp["StrikePrice"]
        Unit = 10000
        index_df.append([date, fund_data.loc[date, "净值"], stock, cash, symbol, expir_price, Unit, option, price])
    index_df = pd.DataFrame(index_df, columns=["date", "stock_price", "50ETF", "Cash", "Symbol", "Strike", "Unit", "Option", "Total"])
    first_swap_date = index_df['date'].iloc[-1]
    return index_df, first_swap_date

In [12]:
def get_pput(index_df):

    # expir_date-1 换仓 重新配置portfolio

    swap_date = index_df['date'].iloc[-1]

    stock_price = fund_data.loc[swap_date, '净值'] #

    threshold = stock_price * 0.95
    Total = index_df.iloc[-1]['Total']


    data = option_data.loc[~option_data['IsCall']].loc[swap_date]
    data = data[data.ExpirationDate == sorted(pd.unique(data.ExpirationDate))[1]]

    tmp = data.loc[data.StrikePrice <= threshold]
    if len(tmp) > 0:
        tmp = tmp.sort_values('StrikePrice').iloc[-1]

    else:
        tmp = data.sort_values('StrikePrice').iloc[0]


    Unit = Total/(stock_price + tmp['Premium'])

    # symbol 改变
    symbol = tmp["InstrumentID"]
    # Unit, symbol改变 换仓完成



    # 开始下一个月的滚动 更新swap_date
    try:
        index = list(pd.unique(option_data.index)).index(str(tmp['ExpirationDate']))
    except:
        index = 0
        print("行权期还未到来")
    new_swap_date = str(pd.unique(option_data.index)[index-1])
    # for date in fund_data.loc[swap_date:expir_date].
    new_df = []
    for date in sorted(list(set(fund_data.loc[swap_date: new_swap_date].iloc[1:].index).intersection(set(option_data.loc[swap_date: new_swap_date].iloc[1:].index)))):
        stock = fund_data.loc[date, "净值"] * Unit
        cash = fund_data.loc[date, "分红"] * Unit
        data = option_data.loc[date]
        tmp = data.loc[data["InstrumentID"] == symbol].iloc[0]
        option = tmp["Premium"] * Unit
        price = stock + option
        expir_price = tmp["StrikePrice"]
        new_df.append([date, fund_data.loc[date, "净值"], stock, cash, symbol, expir_price, Unit, option, price])
    new_df = pd.DataFrame(new_df, columns=["date", "stock_price", "50ETF", "Cash", "Symbol", "Strike", "Unit", "Option", "Total"])
    index_df = pd.concat([index_df, new_df])
    swap_date = new_swap_date
    return index_df, swap_date

In [13]:
def main():
    swap_date_lis = []
    index_df, swap_date = first_day_build(option_data, fund_data)
    swap_date_lis.append(swap_date)

    while pd.to_datetime(swap_date).date()<pd.to_datetime('20211230').date():
        index_df, swap_date = get_pput(index_df)
        swap_date_lis.append(swap_date)
        continue

    index_df["PPUT"] = ((index_df["Total"] + index_df["Cash"]) / index_df["Total"].shift(1)).fillna(1)
    index_df["PPUT"] = index_df["PPUT"].cumprod() * 1000
    index_df["benchmark"] = list(index_df["stock_price"] * 10000)

    index_df['Index'] = (index_df["benchmark"] / index_df["benchmark"].shift(1)).fillna(1)
    index_df['Index'] = index_df["Index"].cumprod() * 1000

    del index_df['benchmark']
    index_df["Index"] = index_df["50ETF"] + index_df["Cash"]/index_df["Unit"] * 10000
    index_df["Index"] = (index_df["Index"] / index_df["Index"].shift(1)).fillna(1)
    index_df["Index"] = index_df["Index"].cumprod() * 1000
    return index_df



In [14]:
pput_df = main()

行权期还未到来


In [15]:
pput_df

,date,stock_price,50ETF,Cash,Symbol,Strike,Unit,Option,Total,PPUT,Index
0,20150209,2.333,23330.000000,0.0,10000006,2.200,10000.00000,617.000000,23947.000000,1000.000000,1000.000000
1,20150210,2.369,23690.000000,0.0,10000006,2.200,10000.00000,451.000000,24141.000000,1008.101224,1015.430776
2,20150211,2.381,23810.000000,0.0,10000006,2.200,10000.00000,321.000000,24131.000000,1007.683635,1020.574368
3,20150212,2.383,23830.000000,0.0,10000006,2.200,10000.00000,224.000000,24054.000000,1004.468201,1021.431633
4,20150213,2.393,23930.000000,0.0,10000006,2.200,10000.00000,159.000000,24089.000000,1005.929762,1025.717960
...,...,...,...,...,...,...,...,...,...,...,...
3,20211227,3.265,35752.199883,0.0,10003720,3.061,10950.13779,51.465647,35803.665530,1656.748976,1532.456060
4,20211228,3.290,36025.953328,0.0,10003720,3.061,10950.13779,33.945428,36059.898756,1668.605699,1544.190027
5,20211229,3.227,35336.094647,0.0,10003720,3.061,10950.13779,108.406364,35444.501010,1640.129297,1514.620431
6,20211230,3.248,35566.047540,0.0,10003720,3.061,10950.13779,62.415786,35628.463327,1648.641816,1524.476963
